Games.01
    1. Minimax Bewertung
        B = min(8,7,3) = 3
        E = max(9,1,6) = 9
        F = max(2, 1, 1) = 2
        G = max(6,5,2) = 6
        C = min(E,F,G) = min(9,2,6) = 2
        D = min(2,1,3) = 1
    2. 
        A -> Alpha(A) =-inf
            B -> kein pruning, Beta(B)=3, Alpha(A)=3
            C -> Beta(C) =+inf, Alpha(A)=3
                E -> kein pruning, Beta(C)=9, Alpha(E)=9
                F -> kein pruning, Beta(C)=2, Alpha(F)=3
                G -> Pruning, da Beta(C)=2, < Alpha(F)=3
            D -> kein pruning, Beta(D)=1, Alpha(A) = 3
    3.
       Da die Bedingung für Pruning nach Auswertung von F erfüllt sind, können F und E getauscht werden, womit dann nicht nur G sondern auch E abgeschnitten wird.


In [ ]:
import math
evaluated_nodes = 0 # For compraison between pruning/no pruning

def create_board():
    return [["" for _ in range(3)] for _ in range(3)] #2-Dimensional Array initialized with empty Strings

def print_board(board):
    for row in board:
        print("|".join([cell if cell != "" else " " for cell in row])) # Print Columns seperated by |
        print("-----") # Seperate Lines
    print()

def check_winner(board):
    # 3 of the Same in a row
    for row in range(3):
        if board[row][0] == board[row][1] == board[row][2] != "":
            return board[row][0]
    # 3 of the Same in a column
    for column in range(3):
        if board[0][column] == board[1][column] == board[2][column] != "":
            return board[0][column]
    # 3 of the Same in a diagonal
    if board[0][0] == board[1][1] == board[2][2] != "":
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] != "":
        return board[0][2]
    return None

# Used to force an end in case of a stalemate
def is_full(board):
    return all(cell != "" for row in board for cell in row)

# Returns 1/-1 for Wins, 0 for Stalemates, False if Game still ongoing
def check_game_over(board):
    winner = check_winner(board)
    if winner=="X":
        return 1
    elif winner=="O":
        return -1
    elif is_full(board):
        return 0
    else:
        return False

## Evaluates and decides on the best available move
def minimax(board, is_max_turn, use_pruning, alpha=-math.inf, beta=math.inf): # Alpha and Beta are either passed from Node above or set to +/- inf
    global evaluated_nodes
    evaluated_nodes += 1

    result = check_game_over(board)
    if result is not False:  # False == Game ongoing
        return result, None  # Return score instead of no move

    # Current best move and associated score
    best_score = -math.inf if is_max_turn else math.inf
    best_move = None

    # Go over all board cells and evaluate possible moves
    for row in range(3):
        for col in range(3):
            if board[row][col] == "": # If Symbol can be placed
                board[row][col] = "X" if is_max_turn else "O" # Place Symbol depending on active player
                score, _ = minimax(board, not is_max_turn, use_pruning, alpha, beta)  # Evaluate recursively (Pass to other player, along with Alpha & Beta)
                board[row][col] = ""  # Remove the Symbol

                # Compare evaluted move to current best move
                if is_max_turn and score > best_score:
                    best_score = score
                    best_move = (row, col)
                    alpha = max(alpha, best_score)
                elif not is_max_turn and score < best_score:
                    best_score = score
                    best_move = (row, col)
                    beta = min(beta, best_score)
                
                # Alpha-Beta pruning, if enabled
                if use_pruning and beta <= alpha:
                    return best_score, best_move


    return best_score, best_move


def play_game(use_pruning):
    # Reset count before each game
    global evaluated_nodes
    evaluated_nodes = 0


    board = create_board()
    is_max_turn = True
    print_board(board)

    while True:
        score, move = minimax(board, is_max_turn, use_pruning)
        row, col = move
        board[row][col] = "X" if is_max_turn else "O"
        print(f"Player {'MAX' if is_max_turn else 'MIN'} plays: {row},{col}")
        print_board(board)

        result = check_game_over(board)
        if result is not False:
            if result == 1:
                print("Player MAX wins")
            elif result == -1:
                print("Player MIN wins")
            else:
                print("It's a stalemate")
            break

        is_max_turn = not is_max_turn  # Switch player

    print(f"Number of evaluated nodes: {evaluated_nodes}") # When game over, show num of evaluted nodes for comparison

# Values for count of evaluted nodes are implausibly high but it still showcases the improvement of pruning
play_game(False)
play_game(True)

Games.03
    Ich habe aus Versehen Minimax direkt in der vereinfachten Version implementiert, da ich mich nicht genau an die Vorlesungsbeispiele gehalten habe und es intuitiv war, die Auswertung von MAX und MIN in einer Funktion zu implementieren. also ist meine Lösung für aufgabe 2 bereits die vereinfachte.

Games.04 : 
    Zustand 1 - Endzustand
        Board:
        X X X
        O O _
        _ _ _
        X3 = 1
        X2 = 0
        X1 = 0
        O3 = 0
        O2 = 1
        O1 = 0
        Eval(s) = +1

    Zustand 2 - Endzustand
        Board:
        O O O
        X X _
        _ _ _

        X3 = 0
        X2 = 1
        X1 = 0
        O3 = 1
        O2 = 0
        O1 = 0
        Eval(s) = -1

    Zustand 3 - Endzustand
        Board:
        X O X
        O X O
        O X O

        X3 = 0
        X2 = 0
        X1 = 0
        O3 = 0
        O2 = 0
        O1 = 0
        Eval(s) = 0

    Zustand 4 :
        Board:
        X X _
        O _ _
        _ _ O

        X2 = 1
        X1 = 1
        O2 = 0
        O1 = 2
        Eval(s) = 3·1 + 1 − (3·0 + 2) = 4

    Zustand 5 :
        Board:
        O O _
        X _ _
        _ X _

        X2 = 0
        X1 = 2
        O2 = 1
        O1 = 0
        Eval(s) = 3·0 + 2 − (3·1 + 0) = −1

    Zustand 6 :
        Board:
        X _ _
        _ O _
        _ _ _

        X2 = 0
        X1 = 2
        O2 = 0
        O1 = 3
        Eval(s) = 3·0 + 2 − (3·0 + 3) = -1

    Einschätzung: 
        Diese Evaluierungsfunktion ist sinnvoll, weil sie:
        - drohende Siege stark gewichtet durch Faktor 3 bei X2 und O2,
        - beide Spieler gleich behandelt (Nullsummenspiel),
        - Positionen, die viele möglichkeiten offenlassen stark gewichtet werden (z.B. die Brettmitte)

Games.05
    Annotierter Baum : minmax-multiplayer_dark.png im selben Ordner